## Nettoyage de données
    
    1) Charger les données depuis le fichier CSV
    
    2) Lister les données pertinentes pour notre application
    
    3) Filtrer les données
    
    4) Nettoyer les données
    



In [1]:
#!pip install plotly

In [2]:
import csv
import math
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

from IPython.display import Image

In [3]:
# Option d'affichage
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)

### 1) Charger les données depuis le fichier CSV

In [ ]:
# Chemin du fichier CSV servant à l'analyse données
path = "C:\Jason\Formation OCR\Projet 2\openfoodfact_dataset.csv"


# Creation du dataset à partir du fichier csv
dataset = pd.read_csv(path, delimiter="\t")
print("Shape du dataset:", dataset.shape, "\n")


# Liste des colonnes disponibles
print("Informations disponibles par produit: ", "\n", dataset.columns.tolist())


In [ ]:
# Affichage des premières lignes du fichier CSV
dataset.head(10)
print("Format du dataset :", dataset.shape, f", soit {dataset.shape[0]} lignes et {dataset.shape[1]} colonnes.")



In [ ]:
# Affichage des valeur manquantes
#msno.bar(dataset)

In [ ]:
# on affiche le nombre de produit par pays
df_countries = dataset["countries_en"].value_counts()
df_countries = df_countries.where(df_countries>1).dropna()

# Création  diagramme des pays les plus représenté
plt.figure(figsize=(10,5))
plt.bar(df_countries.index[:10], df_countries.tolist()[:10])
plt.xlabel('Pays')
plt.ylabel('Fréquence')
plt.title('Diagramme des 10 pays les plus présents dans le dataset')
plt.xticks(rotation=90)
plt.show()

# On va chercher les produits que nous retrouvons en france à l'aide de la variable : countries_en
dataset_france = dataset[dataset["countries_en"] == "France"]
print(f"La France est le pays le plus représenté avec {dataset_france.shape[0]} produits enregistrés.")
print(f"État des données pour la  France:{dataset_france.shape}.")
print(f"Les données sont passées de : {dataset.shape} à {dataset_france.shape}.")
print(f"Soit une suppression de : {dataset.shape[0] - dataset_france.shape[0]}." )

### 2) Lister les données pertinentes pour notre application
     1) Nous prendrons le régime cétogène comme exemple
     2) La variable misc représente les données diverses utiles à notre application
     3) La variable keto représente les variables utiles pour le régime cétogène

In [ ]:
columns_total = dataset.columns.tolist()

# misc correspond aux diverses variable qui pourront nous être utiles. 
misc = ['code', 'product_name', 'image_url', 'nutriscore_score', 'nutriscore_grade', 'energy-kcal_100g']
keto = ['fat_100g', 'saturated-fat_100g', 'monounsaturated-fat_100g',
        'polyunsaturated-fat_100g','omega-3-fat_100g', 'omega-6-fat_100g', 'omega-9-fat_100g', 'trans-fat_100g', 'cholesterol_100g', 'carbohydrates_100g',
        'sugars_100g', 'fiber_100g', 'proteins_100g', 'sodium_100g']

columns = list(dict.fromkeys(misc + keto))
print("Nombre de colonnes conservées:", len(columns), "\n")
print("Liste des colonnes conservées:", "\n", columns, "\n")




In [ ]:
#sns.heatmap(df_keto.loc[:, ~df_keto.columns.isin(['product_name', 'image_url','code', 'nutriscore_grade', 'total_per_100g'])].dropna().corr(), annot=True)

### 3) Filtrer les données

In [ ]:
print("Format du dataset avant filtration des colonnes:", dataset_france.shape, "\n")

# Remplissage des valeurs Nan par 0. Hypothèse : Si il y a des valeurs présente dans certaines variables "_100g" pour un produit et que les autres variables de ce même
# produit sont Nan alors les autres variables pourront être remplis par 0.
# Ensuite, si la somme des variables _100g excepté celle d'energie pour 100g sont égales à 0 alors on supprimera les données
df_keto = dataset_france[columns].copy()

print("Format du dataset après filtration des colonnes et des lignes où les valeurs nutritionnelles n'étaient pas répertoriées:", df_keto.shape, "\n")

### 3) Nettoyer les données 

In [ ]:
# 3.1) Supprimer les lignes vide sans données de valeur nutritionnelles (où la somme des valeurs nutritionnelles par 100g = 0) 
# 3.2) Mettre à jour le type des données
# 3.3) Supprimer les données en double ()
# 3.4) Supprimer les valeurs aberrantes (graph pour visualiser les valeurs aberrantes)



# 3.1) Supprimer les lignes vide sans données de valeur nutritionnelles (où la somme des valeurs nutritionnelles par 100g = 0) 
df_keto[keto] = df_keto[keto].fillna(value=0)
df_keto["total_per_100g"] = df_keto[keto].sum(axis=1)
keto.append("total_per_100g")
df_keto["total_per_100g"].value_counts()


df_keto = df_keto[df_keto.total_per_100g != 0]

df_keto = df_keto.drop("total_per_100g", axis=1)
# Affichage des valeurs manquantes par colonne
msno.matrix(df_keto)




In [ ]:
# 3.2) Mettre à jour le type des données
dtype = {
    "code" : "string",
    "product_name" : "string",
    "image_url" : "string",
    "nutriscore_grade" : "string"}
print("***  Format du dataset *** :", df_keto.shape)
df_keto_type = df_keto.astype(dtype)
print("***  Format du dataset *** :", df_keto_type.shape)
df_keto_type.info()

### 3.2) Supprimer les données en double
    - Lister les produits en double selon les codes barres
    - Supprimer les code barres en double

In [ ]:
def delete_duplicates(df, subset):
    df_cleaned = df.drop_duplicates(subset=subset, keep="first")
    code_duplicate = df["code"].value_counts().index.tolist()
    print("code_duplicate:", code_duplicate[:10])
    index = 1
    print("image_url :", df["image_url"][df["code"] == code_duplicate[index]])
    display(df[df["code"] == code_duplicate[1]])
    display(Image(df["image_url"][df["code"] == code_duplicate[index]].tolist()[0], width=400, height=400))
    if df.shape[0] - df_cleaned.shape[0] > 0:
        print(f"{df.shape[0] - df_cleaned.shape[0]} doublons ont été supprimés")
    else:
        print("Aucun doublons dans le dataset")
    return df_cleaned, df.duplicated(subset=subset)

df_keto_cleaned, df_duplicates = delete_duplicates(df_keto_type, "code")

print("df_keto_cleaned shape : ", df_keto_cleaned.shape)
df_duplicates.head()
#_=msno.matrix(df_keto_type)


In [ ]:

missing_values_sum = df_keto_cleaned.isnull().sum().sort_values()
taux_completion_variable = 100 - df_keto_cleaned.isnull().mean().sort_values()*100

df_completion = pd.DataFrame({"Variable" : taux_completion_variable.index.tolist(), "Taux de completion (%)" : np.round(taux_completion_variable.tolist(), 2) })
print("Le tableau ci-dessous présente les taux de complétions en fonction des variables pour le régime Cétogène")
df_completion


Remarque : 
1) La variable nutriscore est complète à seulement 44.42%, nous viendrons la compléter lors de la phase exploratoire.
2) Les variables product_name et image_url ne sont pas à 100% mais elles ont seulement un rôle descriptifs dans notre étude.


In [ ]:
product_name_counts = df_keto_cleaned["product_name"].value_counts()
y = product_name_counts
product_name = pd.DataFrame({"product_name" : product_name_counts.index.tolist(), "Fréquence" : product_name_counts.tolist()})
product_name


plt.figure(figsize=[14, 7])
col_map = plt.get_cmap('tab20')

# Creating a bar chart from the DataFrame df
pl = plt.bar(product_name["product_name"][:30], product_name["Fréquence"][:30])

for bar in pl:
    plt.annotate(bar.get_height(), 
                 xy=(bar.get_x(), bar.get_height()), 
                     fontsize=10)
plt.xticks(rotation=90)
plt.show()

### 3.3) Suppresion des valeurs aberrantes
 - Utilisation de diagramme à moustache etc
 - Valeur supérieur à 100g pour les valeur nutritionnelle 100g etc
 - afficher la description des valeurs min/max
 - Valeur nutriscore : Les résultats du calcul donnent une valeur comprise entre –15 et +40. La couleur verte correspondant à une valeur comprise entre –15 et –2, le vert clair de –1 à +3, le jaune de +4 à +11, l'orange de +12 à +16 et le rouge de +17 à +40
 

In [ ]:
print("Le tableau ci-dessous présente les statistiques pour chaque variable du régime cétogène")
df_keto_cleaned.describe(percentiles=[]).round()

### Remarque:
    1) Les valeurs du nutriscore sont comprise entre -15 et 40, ce qui démontre qu'il n'y a pas de valeurs aberrantes
    2) Certaines variables nutritionnelles affichent un max supérieur à 100g pour une valeur nutritionnelle par 100g, ce qui est une aberration. 
        Exemple :  fat_100g affiche un max de 2900g par produit de 100g.
    3) L'énergie par 100g en kcal ne peut excéder 900kcal.

Il faut donc procéder à un nettoyage de ces valeurs aberrante

In [ ]:
#remove = ["nutriscore_score", "energy-kcal_100g"]
valeur_nutri = [x for x in df_keto_cleaned.columns.tolist() if x not in misc]

print("Diagramme à moustache représentant la distribution des valeurs pour chaque variable", "\n")
# Création des boîtes à moustache pour détecter les anomalies des variables quantitatives
for var in df_keto_cleaned.columns:
    if df_keto_cleaned[var].dtypes == 'float64':
        sns.boxplot(x=var, data=df_keto_cleaned)
        plt.title(var)
        plt.show()


Le diagramme confirme les valeurs aberrantes présentent dans le tableau descriptif sur les variables :
energy-kcal_100g, fat_100g, saturated-fat_100g, omega-3-fat_100g, omega-9-fat_100g, trans-fat_100g, carbohydrates_100g, sugars_100g, proteins_100g, sodium_100g


In [ ]:
def remove_aberant(df, variable,minimum, maximum):

    data_cleaned = df.copy()
    
    for name in variable:
        if name in data_cleaned.columns.tolist():
            data_cleaned = data_cleaned.query(f'`{name}` <= {maximum}')
            data_cleaned = data_cleaned.query(f'`{name}`>= {minimum}')
            
    return data_cleaned

df_keto_good = remove_aberant(df_keto_cleaned, valeur_nutri, 0, 100)
df_keto_good = remove_aberant(df_keto_good, ["energy-kcal_100g"], 0, 900)
print("Aperçu des statistiques des variables après nettoyage des données aberrantes")
print(f"Il y a eu suppression de {df_keto_cleaned.shape[0] - df_keto_good.shape[0]} lignes dans le dataset.")
df_keto_good.describe()

In [ ]:
print("Aperçu des diagrammes en moustache après nettoyage des données aberrantes")
for var in df_keto_good.columns:
    if df_keto_good[var].dtypes == 'float64':
        sns.boxplot(x=var, data=df_keto_good)
        plt.title(var)
        plt.show()


Notre nettoyage des données est maintenant terminée, la variable nutriscore comporte encore des Nan mais nous tâcherons de combler ces valeurs pendant l'analyse explortatoire.
Voici l'état des données après le nettoyage :


In [ ]:
print(f" Dimension du dataset initial : {dataset.shape}")
print(f" Dimension du dataset final : {df_keto_good.shape}")
df_keto_good.head()

In [ ]:
msno.matrix(df_keto_good)

Exportation des données nettoyées en fichier csv

In [ ]:
df_keto_good.to_csv("C:\Jason\Formation OCR\Projet 2\openfoodfact_dataset_clean.csv")